# CCL and NumCosmo - Learning

First of all, let us import some useful packages.

In [1]:
######## GENERAL ########
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import math

In [2]:
######## NUM COSMO ########
try:
  import gi
  gi.require_version('NumCosmo', '1.0')
  gi.require_version('NumCosmoMath', '1.0')
except:
  pass

from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

In [3]:
######## CCL ########
import pyccl as ccl

Now, let us initialize the NumCosmo library.

In [4]:
#  Initializing the library objects, this must be called before 
#  any other library function.
#
Ncm.cfg_init ()

### Defining the cosmological constants

Here, the values of the cosmological constants are fixed in order to avoid problems when comparing NumCosmo and CCL results.

In [5]:
Omega_c = 0.25   # The cold dark matter density fraction.

Omega_b = 0.05   # The baryonic matter density fraction.

Omega_k = 0.0    # The curvature density fraction.
        
h       = 0.7    # The hubble constant divided by 100 km/s/Mpc; unitless.

A_s     = 2.1e-9 # The power spectrum normalization.

n_s     = 0.96   # The primordial scalar perturbation spectral index.

Neff    = 3.046    # The effective number of massless neutrinos present.

w0 = -1.0        # The first order term of dark energy equation of state.

wa = 0.0         # The second order term of dark energy equation of state.

### Defining the cosmological model

First of all, I will define the CCL cosmological model.

In [6]:
ccl_cosmo = ccl.Cosmology(Omega_c=Omega_c, Omega_b=Omega_b, Neff=Neff,
                          h=h, A_s = A_s, n_s=n_s, Omega_k=Omega_k,
                          w0=w0, wa=wa, transfer_function='eisenstein_hu')

Now, I will define the NumCosmo cosmological model using the same parameters as CCL.

In [7]:
########################################################################
### NcHICosmo — Abstract class for implementing homogeneous and isotropic cosmological models
### NcHICosmoDECpl — Dark Energy -- Chevallier–Polarski–Linder equation of state
### nc_hicosmo_new_from_name (GType parent_type, gchar *cosmo_name): Creates a new instance of "cosmo_name", 
#                                                                    asserting that it descends from "parent_type".
nc_cosmo = Nc.HICosmo.new_from_name (Nc.HICosmo, "NcHICosmoDECpl{'massnu-length':<0>}")

### Source: https://numcosmo.github.io/manual/NcHICosmo.html#nc-hicosmo-new-from-name
### Source: https://numcosmo.github.io/manual/NcHICosmoDECpl.html (Theory)

### massnu-length: this depends on how many neutrinos types we are considering to be massive or massless.
########################################################################



########################################################################
nc_cosmo.omega_x2omega_k ()

### This line is for us to work with Omega_k as a parameter instead of Omega_x (dark energy). If we don't have this
### line, we will automatically be working with Omega_x.
########################################################################



########################################################################
nc_cosmo.param_set_by_name ("Omegac",    ccl_cosmo['Omega_c'])
nc_cosmo.param_set_by_name ("Omegab",    ccl_cosmo['Omega_b'])
#nc_cosmo.param_set_by_name ("Omegax",   0.7)
nc_cosmo.param_set_by_name ("Omegak",    ccl_cosmo['Omega_k'])
nc_cosmo.param_set_by_name ("H0",        ccl_cosmo['h']*100)
nc_cosmo.param_set_by_name ("ENnu",      ccl_cosmo['Neff'])
nc_cosmo.param_set_by_name ("w0",        ccl_cosmo['w0'])
nc_cosmo.param_set_by_name ("w1",        ccl_cosmo['wa'])
########################################################################



Now, I will define the function to perform distance calculations.

In [8]:
########################################################################
### NcDistance — Cosmological distance and time related quantities.
### nc_distance_new (gdouble zf): Creates a new NcDistance object optimized to perform
#                                 distance calculations to redshift up to zf.
dist = Nc.Distance.new (5.0)

### Source: https://numcosmo.github.io/manual/NcDistance.html (Theory)
### Source: https://numcosmo.github.io/manual/NcDistance.html#nc-distance-new
########################################################################



########################################################################
### nc_distance_prepare (NcDistance *dist, NcHICosmo *cosmo):This function prepares the object "dist" using "cosmo",
#                                                            such that all the available distances functions can be
#                                                            evaluated, e.g. nc_distance_comoving().
dist.prepare (nc_cosmo)

### Source: https://numcosmo.github.io/manual/NcDistance.html#nc-distance-prepare
########################################################################

Here, I will define the Transfer Function and the Power Spectrum to be used.

In [9]:
########################################################################
### THIS IS NECESSARY FOR Nc.PowspecMLTransfer.
### NcHIPrim — Abstract class for implementing homogeneous and isotropic primordial cosmological models.
### NcHIPrimPowerLaw — Power law implementation for primordial spectra.

hiprim = Nc.HIPrimPowerLaw.new ()
hiprim.param_set_by_name ("ln10e10ASA", math.log (1.0e10 * ccl_cosmo['A_s']))
hiprim.param_set_by_name ("n_SA", ccl_cosmo['n_s'])
    
nc_cosmo.add_submodel (hiprim)

### Source: https://numcosmo.github.io/manual/NcHIPrimPowerLaw.html (Theory)

########################################################################
### NcTransferFunc — Abstrac class for perturbation transfer function.
### NcTransferFuncEH — Eisenstein-Hu fitting function for the transfer function.
### nc_transfer_func_eh_new (void): Creates a new NcTransferFunc of the NcTransferFuncEH type.

tf_eh = Nc.TransferFuncEH.new ()

### Source: https://numcosmo.github.io/manual/NcTransferFuncEH.html (Theory)
########################################################################

########################################################################
### NcPowspecML — Abstract class for linear matter power spectrum implementation.
### NcPowspecMLTransfer — Class for linear matter power spectrum from a transfer function.
### nc_powspec_ml_transfer_new (NcTransferFunc *tf):Creates a new NcPowspecMLTransfer from the transfer function tf .

ps_eh = Nc.PowspecMLTransfer.new (tf_eh)

### Source: https://numcosmo.github.io/manual/NcPowspecMLTransfer.html (Theory)
########################################################################

########################################################################
ps_eh.prepare (nc_cosmo)
########################################################################

Now, I will define the Mset. In Mset, we keep store a set of models for any posterior calculation.

In [10]:
########################################################################
#NcmMSet — A set of different NcmModel objects.
#NcmModel — Abstract class for implementing models.
mset = Ncm.MSet.empty_new ()
mset.set (nc_cosmo)

# No mset, "guardamos" um conjunto de modelos para cálculos posteriores.
########################################################################

### Getting the data - The data will be used in order to compute the angular power spectrum later